# Moroccan Darija BERT Language Model


## Creating vocab


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

article_lines = []
article_open = False

import string
import re
exclude = set(string.punctuation)

with open('/content/drive/My Drive/arywiki.txt', mode='w', newline='\n', encoding='utf-8') as ofile:
    with open('/content/drive/My Drive/arywiki.xml', mode='r', newline='\n', encoding='utf-8') as file:

        for line in file:
            if '<text' in line:
                article_open = True
            elif '</text>' in line:

                article_open = False
                for oline in article_lines[1:]:
                    if oline != '\n':
                        
                        ofile.write(str(oline).rstrip() + " ")
                ofile.write("\n\n")
                article_lines = []
            else:
                if article_open:
                    line = re.sub(r'[A-Za-z0-9]', ' ' , line)
                    import string
                    line = line.translate(str.maketrans('', '', string.punctuation))
        
                    
                    line = line.rstrip()
                    article_lines.append(line)
                    print (line)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.









             پڭڤبوت بدّل   ڭ ؤ   گ ؤ   ڴ ل ݣ


ستيفن كينݣ مزيود ف    شتنبر      هوّا كاتب ميريكاني ديال حجايةلحجّايات ؤ ريوايةرّيوايات د أداب د لخلعةلخلعة، أداب مورواقعيلأداب لمورواقعي، لأداب د لإجرام ؤ لفانطازيا لكتوبا ديالو تباعو ب كتر من     مليون نسخة،                                                                                        لمبيعات ديال ستيفن كينݣ                 نونبر             ؤ بزاف منهوم ولّاو فيلمفلام، برنامج د تلفازةسيريّةسيريّات د تّلفازة، ؤ حجاية مصورةحجّايات مصورة

بذرة

 عيون لكلام
مراجع





         لفرونسي


                  لفرانساوية







             زدت لپاج






         تصنيفلمريخ
















         بيليروبين











             بوت تغييرات تجميلية



 فيزيولوجي ديال لبيليروبين
ملفتاميطابوليت دلبيليروبين   تصغيرتاميطابوليت دلبيليروبين
لپروصيص دلماكلة دلكواري لحمرين و التحرير ديال لهيمة منهوم ولكاطابوليزان دلهيمة للبيليروبين كايطرا فالطحال، و منبعد 

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3 MB 4.4 MB/s 


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.processors import TemplateProcessing
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents

bert_tokenizer = Tokenizer(WordPiece())
bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])
bert_tokenizer.pre_tokenizer = Whitespace()
bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)

In [ ]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
files = ['/content/drive/My Drive/arywiki.txt']
bert_tokenizer.train(files, trainer)

model_files = bert_tokenizer.model.save("/content/drive/My Drive/data", "bert-wiki")
bert_tokenizer.model = WordPiece.from_file(*model_files, unk_token="[UNK]")

bert_tokenizer.save("/content/drive/My Drive/data/bert-wiki.json")

In [ ]:
### Decoding
output = bert_tokenizer.encode("نسبة نّاس اللي خدامين في لقطاع لخاص")
print(output.ids)


[1, 1724, 1729, 1719, 1759, 1732, 1955, 1942, 2]


In [ ]:
bert_tokenizer.decode([1, 1899, 1914, 1888, 1966, 1917, 2260, 2239, 2])

'الف ##راو ##دين ملف ##قاش حتى يت'

In [ ]:
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

'نسبة ناس اللي خدامين في لقطاع لخاص'

In [ ]:
bert_tokenizer.get_vocab_size()

30522

#2. Train a tokenizer

In [ ]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ['/content/drive/My Drive/arywiki.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model("/content/drive/My Drive/data/", "darijabert")

['/content/drive/My Drive/data/darijabert-vocab.json',
 '/content/drive/My Drive/data/darijabert-merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    '/content/drive/My Drive/data/darijabert-vocab.json',
 '/content/drive/My Drive/data/darijabert-merges.txt',
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)
print(tokenizer.encode("نسبة نّاس اللي خدامين في لقطاع لخاص").tokens)

['<s>', 'ÙĨØ³Ø¨Ø©', 'ĠÙĨ', 'Ùĳ', 'Ø§Ø³', 'ĠØ§ÙĦÙĦÙĬ', 'ĠØ®Ø¯Ø§ÙħÙĬÙĨ', 'ĠÙģÙĬ', 'ĠÙĦÙĤØ·Ø§Ø¹', 'ĠÙĦØ®Ø§Øµ', '</s>']


# 3. Train a language model from scratch




In [ ]:
# Check that we have a GPU
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

False

### We'll define the following config for the model

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 636 kB 63.2 MB/s 
     |████████████████████████████████| 895 kB 75.9 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from google.colab import drive
drive.mount('/content/drive')

tokenizer = ByteLevelBPETokenizer(
            '/content/drive/My Drive/data/darijabert-vocab.json',
 '/content/drive/My Drive/data/darijabert-merges.txt',
        )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 84 million parameters

83504416

## Build our training Dataset


In [ ]:
from transformers import PreTrainedTokenizerFast
# Save the tokenizer you trained
tokenizer.save("byte-level-BPE.tokenizer.json")

# Load it using transformers
tokenizer = PreTrainedTokenizerFast(tokenizer_file="byte-level-BPE.tokenizer.json")

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path= '/content/drive/My Drive/arywiki.txt',
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 7.03 s, sys: 121 ms, total: 7.15 s
Wall time: 2.83 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

## Initialize Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/DarijaBERT',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
print(model.get_output_embeddings())

Linear(in_features=768, out_features=52000, bias=True)


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens({'mask_token': '[MASK]'})

Assigning [PAD] to the pad_token key of the tokenizer
Assigning [MASK] to the mask_token key of the tokenizer


1

### Start training

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
%%time

trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 5335
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 84
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 1h 43min 8s, sys: 1min 52s, total: 1h 45min
Wall time: 52min 40s


TrainOutput(global_step=84, training_loss=5.7579058692568825, metrics={'train_runtime': 3160.7596, 'train_samples_per_second': 1.688, 'train_steps_per_second': 0.027, 'total_flos': 176891443322880.0, 'train_loss': 5.7579058692568825, 'epoch': 1.0})

### Save final model (+ tokenizer + config) to disk

In [ ]:
trainer.save_model("DarijaBERT_model")

Saving model checkpoint to DarijaBERT_model
Configuration saved in DarijaBERT_model/config.json
Model weights saved in DarijaBERT_model/pytorch_model.bin


## 4. Check that the LM actually trained

In [ ]:
from transformers import pipeline
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file="byte-level-BPE.tokenizer.json")
tokenizer.add_special_tokens({'mask_token': '[MASK]'})



mask_fill = pipeline("fill-mask", model="DarijaBERT_model", tokenizer=tokenizer)




Assigning [MASK] to the mask_token key of the tokenizer
loading configuration file DarijaBERT_model/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file DarijaBERT_model/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null

In [ ]:
mask_fill("رسام صانع طباعة{}.".format(tokenizer.mask_token))


[{'score': 0.001710407086648047,
  'sequence': 'رسام صانع طباعة ن.',
  'token': 298,
  'token_str': ' ن'},
 {'score': 0.0006450950168073177,
  'sequence': 'رسام صانع طباعة.',
  'token': 225,
  'token_str': ' '},
 {'score': 0.00032069330336526036,
  'sequence': 'رسام صانع طباعة اللي.',
  'token': 335,
  'token_str': ' اللي'},
 {'score': 0.000285143411019817,
  'sequence': 'رسام صانع طباعة.',
  'token': 49110,
  'token_str': '[MASK]'},
 {'score': 0.00026493999757803977,
  'sequence': 'رسام صانع طباعة كاملة.',
  'token': 2446,
  'token_str': ' كاملة'}]

In [ ]:

mask_fill("كايتعتابر من أساطير لفن لمغريبي حيتاش {}.".format(tokenizer.mask_token))

[{'score': 0.0005538680707104504,
  'sequence': 'كايتعتابر من أساطير لفن لمغريبي حيتاش .',
  'token': 225,
  'token_str': ' '},
 {'score': 0.0004742892342619598,
  'sequence': 'كايتعتابر من أساطير لفن لمغريبي حيتاش  اللي.',
  'token': 335,
  'token_str': ' اللي'},
 {'score': 0.000348238943843171,
  'sequence': 'كايتعتابر من أساطير لفن لمغريبي حيتاش  ف.',
  'token': 292,
  'token_str': ' ف'},
 {'score': 0.0002946967724710703,
  'sequence': 'كايتعتابر من أساطير لفن لمغريبي حيتاش ي.',
  'token': 265,
  'token_str': 'ي'},
 {'score': 0.00029055573395453393,
  'sequence': 'كايتعتابر من أساطير لفن لمغريبي حيتاش               .',
  'token': 454,
  'token_str': '               '}]